# Validation with Pydantic

In [ ]:
class Student:
    def __init__(self, name, age, gender):
        self.name = name
        self.age = age
        self.gender = gender

    # representation av denna instance
    def __repr__(self):
        return f"Student(name = '{self.name}', age = {self.age}, gender = '{self.gender}')"


s1 = Student("John", 37, "M")
s1

Student(name = 'John', age = 37, gender = 'M')

In [ ]:
s2 = Student(3.14444, "femtioåtta", 5)
#detta funkar, men hur skydda?

In [ ]:
#skydda vid input
class Student:
    def __init__(self, name, age, gender):
        self.name = name #attribute
        self.age = age #property
        self.gender = gender #attribute

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if not isinstance(age, (int,float)):
            raise TypeError(f"age must be of type int or float, not {type(age)}")
        if not 0 <= age < 126:
            raise ValueError(f"age must be between 0 or 125, not {age}")
        self._age= age

    # representation av denna instance
    def __repr__(self):
        return f"Student(name = '{self.name}', age = {self.age}, gender = '{self.gender}')"

try:
    Student(3.14, 6, "M")
except TypeError as e:
    print(e)

try:
    Student("adam", -6, "M")
except ValueError as e:
    print(e)

s4 = Student(3.14444, "femtioåtta", 5)


age must be between 0 or 125, not -6


TypeError: age must be of type int or float, not <class 'str'>

## Pydantic

In [11]:
from pydantic import BaseModel, Field, ValidationError
from typing import Literal
class Student(BaseModel):
    name: str
    age: int = Field(gt=-1, lt=125)
    gender: Literal["M", "F"]

try:
    Student(name="John", gender= "M", age= -3)
except ValidationError as e:
    print(e)

1 validation error for Student
age
  Input should be greater than -1 [type=greater_than, input_value=-3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


In [5]:
s1.age = 20
s1.name = "Doddo"
s1

Student(name='Doddo', age=20, gender='M')

In [ ]:
import pandas as pd

df = pd.read_json("housing_data.json")
the_dict= df.to_dict
the_dict

<bound method DataFrame.to_dict of                      address     price  area  rooms
0     Sveavägen 98, Norrmalm   5200000    34    1.5
1   Hornsgatan 72, Södermalm   7600000    52    2.0
2   Värtavägen 12, Östermalm  11200000    78    3.0
3    Götgatan 112, Södermalm   8900000    65    2.5
4  Hälsingegatan 4, Vasastan   6700000    44    1.5>


In [23]:
import json

with open("housing_data.json", 'r') as f:
    json_data = json.loads(f.read())

json_data

[{'address': 'Sveavägen 98, Norrmalm',
  'price': 5200000,
  'area': 34,
  'rooms': 1.5},
 {'address': 'Hornsgatan 72, Södermalm',
  'price': 7600000,
  'area': 52,
  'rooms': 2},
 {'address': 'Värtavägen 12, Östermalm',
  'price': 11200000,
  'area': 78,
  'rooms': 3},
 {'address': 'Götgatan 112, Södermalm',
  'price': 8900000,
  'area': 65,
  'rooms': 2.5},
 {'address': 'Hälsingegatan 4, Vasastan',
  'price': 6700000,
  'area': 44,
  'rooms': 1.5}]

In [24]:
json_data[0]

{'address': 'Sveavägen 98, Norrmalm',
 'price': 5200000,
 'area': 34,
 'rooms': 1.5}

In [25]:
from pydantic import BaseModel

class House(BaseModel):
    address: str
    price: float
    rooms: float

In [30]:
#hur skapa en instance av json_data[0]

#unpackar min key value pairs så går det in i min instanciering.
House(**json_data[0])

House(address='Sveavägen 98, Norrmalm', price=5200000.0, rooms=1.5)

In [31]:
#parsa alla
[House(**house_data) for house_data in json_data]

[House(address='Sveavägen 98, Norrmalm', price=5200000.0, rooms=1.5),
 House(address='Hornsgatan 72, Södermalm', price=7600000.0, rooms=2.0),
 House(address='Värtavägen 12, Östermalm', price=11200000.0, rooms=3.0),
 House(address='Götgatan 112, Södermalm', price=8900000.0, rooms=2.5),
 House(address='Hälsingegatan 4, Vasastan', price=6700000.0, rooms=1.5)]

In [32]:
from pydantic import BaseModel, Field

class House(BaseModel):
    address: str
    price: int = Field(gt=0)
    area: float
    rooms: float

[House(**item) for item in json_data]

[House(address='Sveavägen 98, Norrmalm', price=5200000, area=34.0, rooms=1.5),
 House(address='Hornsgatan 72, Södermalm', price=7600000, area=52.0, rooms=2.0),
 House(address='Värtavägen 12, Östermalm', price=11200000, area=78.0, rooms=3.0),
 House(address='Götgatan 112, Södermalm', price=8900000, area=65.0, rooms=2.5),
 House(address='Hälsingegatan 4, Vasastan', price=6700000, area=44.0, rooms=1.5)]